# RP3Beta and SLIM Elastic Net Hybrid

In [ ]:
# Data import
from utils.data_util import load_URM, create_submission, write_submission
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

# Model
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.DataIO import DataIO

# Hybrid
from utils.Hybrids.MergeModelsByScores import MergeTwoModelsByScoresLinear

from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

# Evaluator
from Evaluation.Evaluator import EvaluatorHoldout

In [ ]:
URM_all = load_URM()

## Split into train and validation set

In [ ]:
URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage=0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage=0.85)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

### Training Vars

In [ ]:
output_folder_path = "../hyper_param_tuning/"

n_cases = 200  # using 10 as an example
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"
cutoff_to_optimize = 10

data_loader = DataIO(folder_path = output_folder_path)

## Load and fit the recommenders

In [ ]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

# Initialize the recommenders
RP3betaRecommender = RP3betaRecommender(URM_train)
SLIMERecommender = SLIMElasticNetRecommender(URM_train)

# Train the RP3betaRecommender recommender with the best hyper tuned values
RP3betaRecommender.fit(
    topK= 81, alpha= 0.7446283980562399, beta= 0.3485498934203173, implicit= True, min_rating= 1
)



# Train the SLIM ElasticNet model with the best hyper tuned values
SLIMERecommender.fit(
    topK= 2781, l1_ratio= 0.03797690559694552, alpha= 0.0014753426395604076
)

## Merge Two Models

In [ ]:
from skopt.space import Real
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs


hyperparameters_range_dictionary = {
    "alpha": Real(0, 1.0),
    "beta": Real(0, 1.0),
}

recommenders = [RP3betaRecommender, SLIMERecommender]

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train, recommenders],     # Data needed in the fit during hyperTuning
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = {},     # Additional hyperparameters for the fit function
)

recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_validation, recommenders],     # Data needed for the last step evaluation
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {},
    EARLYSTOPPING_KEYWORD_ARGS = {},     # Additional hyperparameters for the fit function
)

hyperparameterSearch = SearchBayesianSkopt(MergeTwoModelsByScoresLinear,
                                         evaluator_validation=evaluator_validation,
                                         evaluator_test=evaluator_test)

hyperparameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       hyperparameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "best",
                       output_folder_path = output_folder_path, # Where to save the results
                       output_file_name_root = MergeTwoModelsByScoresLinear.RECOMMENDER_NAME+"_RP3_SLIME_Merge", # How to call the files
                       metric_to_optimize = metric_to_optimize,
                       cutoff_to_optimize = cutoff_to_optimize,
                      )

In [ ]:
search_metadata = data_loader.load_data(MergeTwoModelsByScoresLinear.RECOMMENDER_NAME + "_RP3_SLIME_Merge_metadata.zip")
best_hyperparameters = search_metadata["hyperparameters_best"]
best_hyperparameters

In [ ]:
merged = MergeTwoModelsByScoresLinear(URM_train=URM_train, recommenders=recommenders)
merged.fit(alpha=best_hyperparameters['alpha'], beta=best_hyperparameters['beta'])

In [ ]:
result_df, _ = evaluator_test.evaluateRecommender(merged)
print(result_df.loc[10])

In [ ]:
submission = create_submission(merged)
write_submission(submission=submission, file_name='Hybrid_RP3Beta_SLIME_HT_'+'['+str(result_df.loc[10]['MAP'])+']')